Après le rdv du 05/02, on en sort qu'on veut garder: 
* Une vitesse constante donnée au mur
* Toujours vouloir faire le mouvement le plus rapidement possible. 

On veut changer/ ajouter: 
* On veut rendre le probleme plus complexe en trackant la position du bout de la grue pour faire ce qu'on veut faire. On veut donner la range dans laquelle on peut se trouver pour que le bac suive le mouvement voulu. 
* On veut donner comme limite de glissement le cone de friction du mur qui pivote (A chaque time-step). 


Qu'est-ce qu'on veut minimiser ? 
* 

In [ ]:
import casadi as ca
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

a = 0.300   # m    
b = 0.400   # m
m = 7.0     # kg
mu = 0.3
Fmax = 40.0 # N

Icom = (m/12)*(a*a + b*b)

T = 6.0 
dt = 0.05
N = int(T/dt)

# Rotation of wall 2 with constant speed
phi0 = 0.0
phiT = +np.pi/2
omega = (phiT-phi0)/T
tgrid = np.linspace(0.0, T, N+1)

phi = phi0 + omega * tgrid

pivot = np.array([0.0, 0.0], dtype=float) #pivot position 

def RM(theta):
    c, s = ca.cos(theta), ca.sin(theta)
    return ca.vertcat(
        ca.horzcat(c, -s),
        ca.horzcat(s,  c)
    )

def wall_vectors(phi):
    t = ca.vertcat(-ca.sin(phi), ca.cos(phi))  # tangent
    n = ca.vertcat( ca.cos(phi), ca.sin(phi))  # normal
    return t, n

t_vec, n_vec = wall_vectors(phi)

opti = ca.Opti()

# States
xC = opti.variable(N+1)
yC = opti.variable(N+1)
th = opti.variable(N+1)
vx = opti.variable(N+1)
vy = opti.variable(N+1)
om = opti.variable(N+1)


# Input
x_g = opti.variable(N)
y_g = opti.variable(N)

# Contact forces of the wall
lam_n = opti.variable(N)
lam_t = opti.variable(N)


opti.subject_to(lam_n >= 0)

t0, n0 = wall_vectors_np(phi0)

d_plus  = n0 + mu * t0
d_minus = n0 - mu * t0

# normaliser juste pour l'affichage
d_plus  = d_plus / np.linalg.norm(d_plus)
d_minus = d_minus / np.linalg.norm(d_minus)
n0u = n0 / np.linalg.norm(n0)
t0u = t0 / np.linalg.norm(t0)

plt.figure()
plt.axhline(0); plt.axvline(0)
plt.quiver(0,0, n0u[0], n0u[1], angles='xy', scale_units='xy', scale=1, label='n')
plt.quiver(0,0, t0u[0], t0u[1], angles='xy', scale_units='xy', scale=1, label='t')
plt.quiver(0,0, d_plus[0], d_plus[1], angles='xy', scale_units='xy', scale=1, label='cone edge +')
plt.quiver(0,0, d_minus[0], d_minus[1], angles='xy', scale_units='xy', scale=1, label='cone edge -')

plt.title("Friction cone directions at k=0 (phi=0)")
plt.axis("equal"); plt.grid(True); plt.legend()
plt.show()



